# Table of contents:

* [Introduction to block ciphers](#intro-block)
* [Padding a message](#message-padding)
* [The Advanced Encryption Standard (AES)](#AES)
* [Modes of operation of block ciphers](#modes)
* [Size of the output ciphertex on AES](#size)
* [Bonus: Fernet cipher](#fernet)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.



## Block Ciphers <a class="anchor" id="intro-block"></a>

Block ciphers as opposed to stream ciphers take a block of the plaintext (a specific amount of bytes) and encrypts it into a block with the same size. In this section we will use the Advanced Encryption Standard (AES) to understand block ciphers. In the next schema it is shown how an original message of arbitrary $N$ bytes is converted into a ciphertext having blocks of $K$ bytes. The ciphertext size is always a multiple of $K$ bytes.

## Padding a message <a class="anchor" id="message-padding"></a>

Most of the times the lenght of the message is not a multiple of the block size so we need to "pad" the message to have the required length. A common padding function is [PKCS7](https://en.wikipedia.org/wiki/Padding_(cryptography)). Basically what PKCS7 does is appendinng a list of bytes with the same value corresponding to the number of bytes needed to complete the block.




## Encrypting using AES (Advanced Encryption Standard) <a class="anchor" id="AES"></a>

AES is a block cipher that was established as a standard by NIST in 2001 (after a public call to improve/substitute DES encryption algorithm in 1997). AES is a subset of the Rijndael block cipher developed by Vincent Rijmen and Joan Daemen submitted to NIST during the [AES selection process](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard_process).


We are not going to go into the details of te exact implementation but the readers are referred to the book of [Katz and Lindell](http://www.cs.umd.edu/~jkatz/imc.html) Chapter 6 section 2. Also Mike Pound explains AES in this [video](https://www.youtube.com/watch?v=O4xNJsjtN6E&t=524s&ab_channel=Computerphile), check it out!

In [1]:
%pip install pycrypto
%pip install pycryptodome
%pip install cryptography

  Using cached pycrypto-2.6.1.tar.gz (446 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for pycrypto, since package 'wheel' is not installed.
  Running setup.py install for pycrypto: started
  Running setup.py install for pycrypto: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Running setup.py install for pycrypto did not run successfully.
  │ exit code: 1
  ╰─> [181 lines of output]
      running install
      c:\Python310\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\Crypto
      copying lib\Crypto\pct_warnings.py -> build\lib.win-amd64-cpython-310\Crypto
      copying lib\Crypto\__init__.py -> build\lib.win-amd64-cpython-310\Crypto
      creating build\lib.win-amd64-cpython-310\Crypto\Hash
      copying lib\Crypto\Hash\hashalgo.py -> build\lib.win-amd64-cpython-310\Crypto\Hash
      copying lib\Crypto\Hash\HMAC.py -> build\lib.win-amd64-cpython-310\Crypto\Hash
      copying lib\Crypto\Hash\MD2.py -> build\lib.win-amd

Note: you may need to restart the kernel to use updated packages.


In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode

key=b'1234567891234567'
nonce = get_random_bytes(15)    # instead of getting a random nonce
message = "Information security is a great course !".encode() # convert the string to bytes using encode function

# Encryption
cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
ciphertext, mac = cipher.encrypt_and_digest(message)
# Decryption
cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
plaintext = cipher.decrypt_and_verify(ciphertext, mac)

print("PlainText before Encryption: "+message.decode())
print("Ciphertext: "+b64encode(ciphertext).decode())
print("Back to PlainText: "+plaintext.decode())

PlainText before Encryption: Information security is a great course !
Ciphertext: iqegaosc2U9dP/mHA1duYl3DlDSFYHkLEBJ5DZ8bKk2cKQRQ5fakUQ==
Back to PlainText: Information security is a great course !


## Modes of operation of block ciphers <a class="anchor" id="mode"></a>

A block cipher by itself is only suitable for the secure cryptographic transformation (encryption or decryption) of one fixed-length group of bits called a block. A mode of operation describes how to repeatedly apply a cipher's single-block operation to securely transform amounts of data larger than a block ([Wikipedia](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation)).

The first mode is "not doing anything", this is the Electronic Codebook mode. See the figure below (from Wikipedia).
We are lucky and in ```cryptography``` package ECB implemented in ```cryptography.hazmat.primitives.ciphers.ECB``` function (we've seen in the previous example!).
Now we can encrypt the same message twice and see what we get in the ciphertext:

# AES USING ECB (ELECTRONIC CODE BOOK)

In [3]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode
for x in range(2):
    # key = get_random_bytes(16)      # A 16 byte key for AES-128
    key=b'1234567891234567'
    message = "Information security is a great course, I love it !".encode() # convert the string to bytes using encode function

    # Encryption
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext= cipher.encrypt(pad(message, AES.block_size)) # Block size= 16 bytes
    # Decryption
    decrypt_cipher = AES.new(key, AES.MODE_ECB)
    plaintext =  decrypt_cipher.decrypt(ciphertext)
    print("Ciphertext "+str(x)+" : "+b64encode(ciphertext).decode())
    print("Back to PlainText "+str(x)+" : "+plaintext.decode()+'\n')

Ciphertext 0 : TLHenMawihtNigFCM8Ub0mDIBGYS4iN33JaUOLJ6/xeW9hrNVPnMM6ooHIZdZH9P3JUNHQEy2+UwjdMz8WQNmA==
Back to PlainText 0 : Information security is a great course, I love it !

Ciphertext 1 : TLHenMawihtNigFCM8Ub0mDIBGYS4iN33JaUOLJ6/xeW9hrNVPnMM6ooHIZdZH9P3JUNHQEy2+UwjdMz8WQNmA==
Back to PlainText 1 : Information security is a great course, I love it !



This is not a desirable outcome. If I want to send the same message twice, I really don't want to send the same ciphertext. What if in all comunications I start by "Dear..." and the attacker knows it?. A better mode is the Cipher block chaining (CBC):

# AES USING CBC (CIPHER BLOCK CHAINING)

In [4]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode
for x in range(2):
    # key = get_random_bytes(16)      # A 16 byte key for AES-128
    key=b'1234567891234567'
    message = "Information Security is the best course, I love it !".encode() # convert the string to bytes using encode function

    # Encryption
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext= cipher.encrypt(pad(message, AES.block_size)) # Block size= 16 bytes
    iv = cipher.iv
    # Decryption
    decrypt_cipher = AES.new(key, AES.MODE_CBC,iv)
    plaintext =  decrypt_cipher.decrypt(ciphertext)
    print("Ciphertext "+str(x)+" : "+b64encode(ciphertext).decode())
    print("Back to PlainText "+str(x)+" : "+plaintext.decode()+'\n')

Ciphertext 0 : b5CzWLMWhc3J5re1QYzMcTDF68/ROYXB269PE+NCMKqQaFVh8Jl0qBWZaBBbVHEickv6lS5kgavE7qq9uzwTrg==
Back to PlainText 0 : Information Security is the best course, I love it !

Ciphertext 1 : ByZ3X/9rL7mmu3pZp0jVk0VPtQCzvOyZogmGB4xr++sRRZ+EDvX6pFqq98i5ocSwS0fmDZ8y7bmvD21YUeoiSw==
Back to PlainText 1 : Information Security is the best course, I love it !



In this case we take a random initialization vector and perform XOR operation with the block of plaintext, then we feed this into the encryptor, after that we obtain the ciphertext. This ciphertext is used as the initialization vector to encrypt the next block.

## Size of ciphertext <a class="anchor" id="size"></a>

In [5]:
print("Length of CipherText: "+str(len(ciphertext)))
print("Length of PlainText: "+str(len(plaintext)))

Length of CipherText: 64
Length of PlainText: 64


The ciphertext has a length of 64 , same as the plaintext

## Bonus: Fernet <a class="anchor" id="fernet"></a>

Another block cipher implemented in cryptography package is [Fernet](https://asecuritysite.com/encryption/fernet). 

In [6]:
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import sys
import binascii
import base64

password="hello"
val="hello world"
def get_key(password):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(password)
    return base64.urlsafe_b64encode(digest.finalize())

if (len(sys.argv)>1):
	val=sys.argv[1]

if (len(sys.argv)>2):
	password=str(sys.argv[2])

if (len(password)>1):
	key = get_key(password.encode())
else:
	key = Fernet.generate_key()

print("Key: ",binascii.hexlify(bytearray(key)))
cipher_suite = Fernet(key)
cipher_text = cipher_suite.encrypt(val.encode())
cipher=binascii.hexlify(bytearray(cipher_text))
print("Cipher: ",cipher)
print("\nVersion:\t",cipher[0:2])
print("Time stamp:\t",cipher[2:18])
print("IV:\t\t",cipher[18:50])
print("HMAC:\t\t",cipher[-64:])
plain_text = cipher_suite.decrypt(cipher_text)
print("\nPlain text: ",plain_text)

Key:  b'5f734f685251552d4f384245644c394e4946526266632d355468434b74706d34676c66493155424a662d733d'
Cipher:  b'674141414141426a643837597039384e566a51726d4a6f507339686c75746d5f536a7a6342702d787147496e4c5f336c3859565f6541525475346a67693572544e744d4447723141505f426c466750334864574e44344739647a3664726672476a673d3d'

Version:	 b'67'
Time stamp:	 b'4141414141426a64'
IV:		 b'3837597039384e566a51726d4a6f5073'
HMAC:		 b'47723141505f426c466750334864574e44344739647a3664726672476a673d3d'

Plain text:  b'--ip=127.0.0.1'
